# Python Advanced

## Decorator fixSeed

In [167]:
# 固定 seed = 0 版本

import numpy as np
import functools

def fixSeed(fnc):
    @functools.wraps(fnc)
    def wrapperFixSeed(*args, **kargs):
        np.random.seed(0)
        returnValue = fnc(*args, **kargs)
        np.random.seed()
        return returnValue
    return wrapperFixSeed

@fixSeed
def printRand():
    print(np.random.uniform())
    
printRand()
print(np.random.uniform())

0.5488135039273248
0.4123412361846971


In [171]:
# 接受 argument 版本，但變成一定要指定 seed

import numpy as np
import functools

def fixSeed(seed=0):
    def decoratorFixSeed(fnc):
        @functools.wraps(fnc)
        def wrapperFixSeed(*args, **kargs):
            np.random.seed(seed)
            returnValue = fnc(*args, **kargs)
            np.random.seed()
            return returnValue
        return wrapperFixSeed
    return decoratorFixSeed

@fixSeed(100)
def printRand():
    print(np.random.uniform())
    
printRand()
print(np.random.uniform())

0.5434049417909654
0.7994076348626472
